# Compare models in a hybrid setup: Motion feedback- Force actuation

- numerical system: rotor aerodynamic simulator; calculate rotor loads
- physical turbine: measure motions and accelerations
- Ml model: multi-output regressor

In [1]:
from fowt_ml.pipeline import Pipeline

In [2]:
example_config_file = "../../src/example_config.yml"

In [3]:
my_pipeline = Pipeline(example_config_file)

In [4]:
# set correct path for mat file: add the correct path
my_pipeline.data_config["exp699"]["path_file"] = "/home/sarah/temp/hybridlabs/data_example/exp699.mat"

In [5]:
# check the targets and predictors that are provided in the comfig
print("targets are:", my_pipeline.target_labels)
print("features are:", my_pipeline.predictors_labels)

targets are: ['force_aero_est6[0]', 'force_aero_est6[1]', 'force_aero_est6[2]', 'force_aero_est6[3]', 'force_aero_est6[4]', 'force_aero_est6[5]', 'force_tt_meas6[0]', 'force_tt_meas6[1]', 'force_tt_meas6[2]', 'force_tt_meas6[3]', 'force_tt_meas6[4]', 'force_tt_meas6[5]']
features are: ['acc_tb_meas3[0]', 'acc_tb_meas3[1]', 'acc_tb_meas3[2]', 'acc_tt_meas3[0]', 'acc_tt_meas3[1]', 'acc_tt_meas3[2]', 'pos_act6[0]', 'pos_act6[1]', 'pos_act6[2]', 'pos_act6[3]', 'pos_act6[4]', 'pos_act6[5]', 'spd_rot_act', 'wind_speed']


In [6]:
# check the models and metrics that are provided by the config
print(my_pipeline.model_names)
print(my_pipeline.metric_names)

{'ElasticNetRegression': {}, 'LeastAngleRegression': {}, 'LassoRegression': {}, 'LinearRegression': {}, 'SklearnGPRegressor': {'num_inducing': 100, 'num_latents': 3, 'num_epochs': 1}, 'RandomForest': {'n_estimators': 50, 'max_depth': 9, 'bootstrap': True, 'max_samples': 10000}, 'MultilayerPerceptron': {'hidden_layer_sizes': 10, 'max_iter': 10}, 'XGBoostRegression': {'n_estimators': 100, 'max_depth': 4, 'tree_method': 'hist'}}
['neg_mean_squared_error', 'neg_root_mean_squared_error', 'r2', 'model_fit_time', 'model_predict_time']


In [7]:
# setup the pipeline
my_pipeline.setup(data="exp699")

In [8]:
# compare models on test data
models, scores = my_pipeline.compare_models(sort="model_fit_time")

/home/sarah/miniconda3/envs/fowt_ml/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [9]:
scores

,neg_mean_squared_error,neg_root_mean_squared_error,r2,model_fit_time,model_predict_time
LeastAngleRegression,-7.127108,-2.259902,0.402951,0.145,0.001
LinearRegression,-7.122590,-2.259118,0.403461,0.277,0.001
ElasticNetRegression,-7.546190,-2.326244,0.361426,6.665,0.001
MultilayerPerceptron,-7.390511,-2.324204,0.274377,6.673,0.001
LassoRegression,-7.762648,-2.360468,0.344400,7.874,0.001
RandomForest,-7.016881,-2.228596,0.428391,14.582,0.007
XGBoostRegression,-5.965616,-2.049794,0.509560,15.832,0.002
SklearnGPRegressor,-26.811133,-3.971338,-0.411956,27.892,0.006


In [10]:
models

{'ElasticNetRegression': ElasticNet(),
 'LeastAngleRegression': Lars(),
 'LassoRegression': Lasso(),
 'LinearRegression': LinearRegression(),
 'SklearnGPRegressor': SklearnGPRegressor(num_epochs=1, num_inducing=100, num_latents=3),
 'RandomForest': RandomForestRegressor(max_depth=9, max_samples=10000, n_estimators=50),
 'MultilayerPerceptron': MLPRegressor(hidden_layer_sizes=10, max_iter=10),
 'XGBoostRegression': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None

In [11]:
# compare models using cross_validation
models, scores = my_pipeline.compare_models(sort="model_predict_time", cross_validation=True)

/home/sarah/miniconda3/envs/fowt_ml/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/sarah/miniconda3/envs/fowt_ml/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/sarah/miniconda3/envs/fowt_ml/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/sarah/miniconda3/envs/fowt_ml/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  war

In [12]:
scores

,model_fit_time,neg_mean_squared_error,neg_root_mean_squared_error,r2,model_predict_time
ElasticNetRegression,3.799000,-7.609064,-2.336870,0.362144,0.001000
LeastAngleRegression,0.159000,-7.196305,-2.271585,0.402931,0.001000
LinearRegression,0.293667,-7.190567,-2.270583,0.403588,0.001000
MultilayerPerceptron,4.338333,-8.186005,-2.493574,-0.387894,0.001000
XGBoostRegression,10.858667,-6.009705,-2.057880,0.510675,0.002000
LassoRegression,3.871000,-7.824502,-2.371108,0.345111,0.002333
RandomForest,14.062667,-7.046162,-2.233325,0.431115,0.005333
SklearnGPRegressor,18.646333,-31.044421,-4.059340,-0.083243,0.006000
